In [4]:
import subprocess
import sys
import os

def install(package):
    """Auto-install missing packages"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])
        print(f"✓ Installed {package}")
        return True
    except Exception as e:
        print(f"✗ Failed to install {package}: {e}")
        return False

# Required packages
packages = {
    'pandas': 'pandas',
    'numpy': 'numpy', 
    'matplotlib': 'matplotlib',
    'seaborn': 'seaborn',
    'pillow': 'PIL',
    'scikit-learn': 'sklearn',
    'tensorflow': 'tensorflow',
    'opencv-python': 'cv2',
    'wordcloud': 'wordcloud',
    'nltk': 'nltk'
}

print("🔧 Checking dependencies...")
for name, import_name in packages.items():
    try:
        __import__(import_name)
    except ImportError:
        print(f"Installing {name}...")
        install(name)

# Core imports
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import PCA, LatentDirichletAllocation
import glob
from collections import Counter
import re

# NLP imports
try:
    import nltk
    from wordcloud import WordCloud
    nltk.download('stopwords', quiet=True)
    nltk.download('punkt', quiet=True)
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize
    NLP_AVAILABLE = True
except:
    NLP_AVAILABLE = False
    print("⚠ NLP libraries limited")

# Deep Learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import ResNet50, VGG16, InceptionV3
    from tensorflow.keras.preprocessing.image import load_img, img_to_array
    from tensorflow.keras.models import Sequential, Model
    from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
    from tensorflow.keras.optimizers import Adam
    DL_AVAILABLE = True
except:
    DL_AVAILABLE = False
    print("⚠ Deep learning limited")

# OpenCV
try:
    import cv2
    CV_AVAILABLE = True
except:
    CV_AVAILABLE = False

# =====================================================
# CONFIGURATION
# =====================================================
FAKE_PATH = r"C:\Users\dibya\OneDrive\Desktop\miniproj\Fake.csv"
TRUE_PATH = r"C:\Users\dibya\OneDrive\Desktop\miniproj\True.csv"
REDDIT_PATH = r"C:\Users\dibya\OneDrive\Desktop\miniproj\fakeddit_subset"
MEDICAL_PATH = r"C:\Users\dibya\OneDrive\Desktop\miniproj\snopes_medical"

# Create output directory
os.makedirs('eda_outputs', exist_ok=True)

print("\n" + "="*80)
print("🔬 ADVANCED EDA - MULTIMODAL FAKE NEWS DETECTION")
print("="*80)

# =====================================================
# PHASE 1: ADVANCED TEXT ANALYSIS
# =====================================================
print("\n" + "="*80)
print("[PHASE 1] ADVANCED TEXT DATA ANALYSIS")
print("="*80)

df_text = None
df_fake = None
df_true = None

try:
    if os.path.exists(FAKE_PATH) and os.path.exists(TRUE_PATH):
        df_fake = pd.read_csv(FAKE_PATH)
        df_fake['label'] = 0
        df_fake['category'] = 'Fake'
        
        df_true = pd.read_csv(TRUE_PATH)
        df_true['label'] = 1
        df_true['category'] = 'True'
        
        df_text = pd.concat([df_fake, df_true], ignore_index=True)
        print(f"✓ Loaded {len(df_text):,} articles ({len(df_fake):,} fake, {len(df_true):,} true)")
        print(f"  Columns: {df_text.columns.tolist()}")
        
        # ===== 1.1 TEXT LENGTH ANALYSIS =====
        print("\n📏 Text Length Analysis:")
        if 'text' in df_text.columns:
            df_text['text_length'] = df_text['text'].astype(str).apply(len)
            df_text['word_count'] = df_text['text'].astype(str).apply(lambda x: len(x.split()))
            
            # Add to individual dataframes too
            df_fake['text_length'] = df_fake['text'].astype(str).apply(len)
            df_fake['word_count'] = df_fake['text'].astype(str).apply(lambda x: len(x.split()))
            df_true['text_length'] = df_true['text'].astype(str).apply(len)
            df_true['word_count'] = df_true['text'].astype(str).apply(lambda x: len(x.split()))
            
            print(f"\nFake News Statistics:")
            print(f"  Avg length: {df_fake['text_length'].mean():.0f} chars")
            print(f"  Avg words: {df_fake['word_count'].mean():.0f}")
            
            print(f"\nTrue News Statistics:")
            print(f"  Avg length: {df_true['text_length'].mean():.0f} chars")
            print(f"  Avg words: {df_true['word_count'].mean():.0f}")
            
            # Visualization
            fig, axes = plt.subplots(2, 2, figsize=(16, 12))
            
            # Text length distribution
            axes[0, 0].hist(df_fake['text_length'], bins=50, alpha=0.6, label='Fake', color='red', edgecolor='black')
            axes[0, 0].hist(df_true['text_length'], bins=50, alpha=0.6, label='True', color='green', edgecolor='black')
            axes[0, 0].set_xlabel('Text Length (characters)')
            axes[0, 0].set_ylabel('Frequency')
            axes[0, 0].set_title('Text Length Distribution', fontweight='bold', fontsize=12)
            axes[0, 0].legend()
            axes[0, 0].grid(alpha=0.3)
            
            # Word count distribution
            axes[0, 1].hist(df_fake['word_count'], bins=50, alpha=0.6, label='Fake', color='red', edgecolor='black')
            axes[0, 1].hist(df_true['word_count'], bins=50, alpha=0.6, label='True', color='green', edgecolor='black')
            axes[0, 1].set_xlabel('Word Count')
            axes[0, 1].set_ylabel('Frequency')
            axes[0, 1].set_title('Word Count Distribution', fontweight='bold', fontsize=12)
            axes[0, 1].legend()
            axes[0, 1].grid(alpha=0.3)
            
            # Box plots
            data_to_plot = [df_fake['text_length'], df_true['text_length']]
            axes[1, 0].boxplot(data_to_plot, labels=['Fake', 'True'], patch_artist=True,
                              boxprops=dict(facecolor='lightblue'))
            axes[1, 0].set_ylabel('Text Length (characters)')
            axes[1, 0].set_title('Text Length Comparison', fontweight='bold', fontsize=12)
            axes[1, 0].grid(alpha=0.3)
            
            data_to_plot = [df_fake['word_count'], df_true['word_count']]
            axes[1, 1].boxplot(data_to_plot, labels=['Fake', 'True'], patch_artist=True,
                              boxprops=dict(facecolor='lightcoral'))
            axes[1, 1].set_ylabel('Word Count')
            axes[1, 1].set_title('Word Count Comparison', fontweight='bold', fontsize=12)
            axes[1, 1].grid(alpha=0.3)
            
            plt.tight_layout()
            plt.savefig('eda_outputs/text_length_analysis.png', dpi=300, bbox_inches='tight')
            plt.close()
            print("✓ Saved: eda_outputs/text_length_analysis.png")
        
        # ===== 1.2 TITLE ANALYSIS =====
        if 'title' in df_text.columns:
            print("\n📰 Title Analysis:")
            df_text['title_length'] = df_text['title'].astype(str).apply(len)
            df_text['title_word_count'] = df_text['title'].astype(str).apply(lambda x: len(x.split()))
            df_text['has_question'] = df_text['title'].astype(str).str.contains('\?', regex=True)
            df_text['has_exclamation'] = df_text['title'].astype(str).str.contains('!', regex=False)
            df_text['all_caps_words'] = df_text['title'].astype(str).apply(lambda x: sum(1 for word in x.split() if word.isupper() and len(word) > 2))
            
            # Add to individual dataframes too
            df_fake['title_length'] = df_fake['title'].astype(str).apply(len)
            df_fake['title_word_count'] = df_fake['title'].astype(str).apply(lambda x: len(x.split()))
            df_fake['has_question'] = df_fake['title'].astype(str).str.contains('\?', regex=True)
            df_fake['has_exclamation'] = df_fake['title'].astype(str).str.contains('!', regex=False)
            df_fake['all_caps_words'] = df_fake['title'].astype(str).apply(lambda x: sum(1 for word in x.split() if word.isupper() and len(word) > 2))
            
            df_true['title_length'] = df_true['title'].astype(str).apply(len)
            df_true['title_word_count'] = df_true['title'].astype(str).apply(lambda x: len(x.split()))
            df_true['has_question'] = df_true['title'].astype(str).str.contains('\?', regex=True)
            df_true['has_exclamation'] = df_true['title'].astype(str).str.contains('!', regex=False)
            df_true['all_caps_words'] = df_true['title'].astype(str).apply(lambda x: sum(1 for word in x.split() if word.isupper() and len(word) > 2))
            
            print(f"\nFake News Titles:")
            print(f"  Avg length: {df_fake['title_length'].mean():.1f} chars")
            print(f"  Questions: {(df_fake['has_question'].sum() / len(df_fake) * 100):.1f}%")
            print(f"  Exclamations: {(df_fake['has_exclamation'].sum() / len(df_fake) * 100):.1f}%")
            
            print(f"\nTrue News Titles:")
            print(f"  Avg length: {df_true['title_length'].mean():.1f} chars")
            print(f"  Questions: {(df_true['has_question'].sum() / len(df_true) * 100):.1f}%")
            print(f"  Exclamations: {(df_true['has_exclamation'].sum() / len(df_true) * 100):.1f}%")
            
            # Visualization
            fig, axes = plt.subplots(2, 2, figsize=(16, 10))
            
            # Title length
            axes[0, 0].hist(df_fake['title_length'], bins=30, alpha=0.6, label='Fake', color='red')
            axes[0, 0].hist(df_true['title_length'], bins=30, alpha=0.6, label='True', color='green')
            axes[0, 0].set_xlabel('Title Length')
            axes[0, 0].set_ylabel('Frequency')
            axes[0, 0].set_title('Title Length Distribution', fontweight='bold')
            axes[0, 0].legend()
            axes[0, 0].grid(alpha=0.3)
            
            # Punctuation usage
            punct_data = pd.DataFrame({
                'Category': ['Fake', 'True', 'Fake', 'True'],
                'Type': ['Questions', 'Questions', 'Exclamations', 'Exclamations'],
                'Percentage': [
                    (df_fake['has_question'].sum() / len(df_fake) * 100),
                    (df_true['has_question'].sum() / len(df_true) * 100),
                    (df_fake['has_exclamation'].sum() / len(df_fake) * 100),
                    (df_true['has_exclamation'].sum() / len(df_true) * 100)
                ]
            })
            
            x = np.arange(2)
            width = 0.35
            axes[0, 1].bar(x - width/2, punct_data[punct_data['Category'] == 'Fake']['Percentage'], 
                          width, label='Fake', color='red', alpha=0.7)
            axes[0, 1].bar(x + width/2, punct_data[punct_data['Category'] == 'True']['Percentage'], 
                          width, label='True', color='green', alpha=0.7)
            axes[0, 1].set_ylabel('Percentage')
            axes[0, 1].set_title('Punctuation Usage in Titles', fontweight='bold')
            axes[0, 1].set_xticks(x)
            axes[0, 1].set_xticklabels(['Questions (?)', 'Exclamations (!)'])
            axes[0, 1].legend()
            axes[0, 1].grid(alpha=0.3)
            
            # All caps words
            axes[1, 0].hist(df_fake['all_caps_words'], bins=20, alpha=0.6, label='Fake', color='red')
            axes[1, 0].hist(df_true['all_caps_words'], bins=20, alpha=0.6, label='True', color='green')
            axes[1, 0].set_xlabel('Number of ALL CAPS Words')
            axes[1, 0].set_ylabel('Frequency')
            axes[1, 0].set_title('ALL CAPS Usage', fontweight='bold')
            axes[1, 0].legend()
            axes[1, 0].grid(alpha=0.3)
            
            # Title word count
            axes[1, 1].hist(df_fake['title_word_count'], bins=20, alpha=0.6, label='Fake', color='red')
            axes[1, 1].hist(df_true['title_word_count'], bins=20, alpha=0.6, label='True', color='green')
            axes[1, 1].set_xlabel('Title Word Count')
            axes[1, 1].set_ylabel('Frequency')
            axes[1, 1].set_title('Title Word Count Distribution', fontweight='bold')
            axes[1, 1].legend()
            axes[1, 1].grid(alpha=0.3)
            
            plt.tight_layout()
            plt.savefig('eda_outputs/title_analysis.png', dpi=300, bbox_inches='tight')
            plt.close()
            print("✓ Saved: eda_outputs/title_analysis.png")
        
        # ===== 1.3 WORD CLOUD ANALYSIS =====
        if NLP_AVAILABLE and 'text' in df_text.columns:
            print("\n☁️ Generating Word Clouds...")
            
            stop_words = set(stopwords.words('english'))
            
            # Fake news word cloud
            fake_text = ' '.join(df_fake['text'].astype(str).tolist())
            fake_text = re.sub(r'[^\w\s]', '', fake_text.lower())
            
            # True news word cloud
            true_text = ' '.join(df_true['text'].astype(str).tolist())
            true_text = re.sub(r'[^\w\s]', '', true_text.lower())
            
            fig, axes = plt.subplots(1, 2, figsize=(20, 8))
            
            wordcloud_fake = WordCloud(width=800, height=400, background_color='white', 
                                      stopwords=stop_words, colormap='Reds').generate(fake_text)
            axes[0].imshow(wordcloud_fake, interpolation='bilinear')
            axes[0].set_title('Fake News Word Cloud', fontweight='bold', fontsize=16)
            axes[0].axis('off')
            
            wordcloud_true = WordCloud(width=800, height=400, background_color='white', 
                                      stopwords=stop_words, colormap='Greens').generate(true_text)
            axes[1].imshow(wordcloud_true, interpolation='bilinear')
            axes[1].set_title('True News Word Cloud', fontweight='bold', fontsize=16)
            axes[1].axis('off')
            
            plt.tight_layout()
            plt.savefig('eda_outputs/wordclouds.png', dpi=300, bbox_inches='tight')
            plt.close()
            print("✓ Saved: eda_outputs/wordclouds.png")
        
        # ===== 1.4 TOP N-GRAMS ANALYSIS =====
        if 'text' in df_text.columns:
            print("\n📊 N-Gram Analysis...")
            
            # Bigrams
            vectorizer_bi = CountVectorizer(ngram_range=(2, 2), max_features=20, stop_words='english')
            
            fake_bigrams = vectorizer_bi.fit_transform(df_fake['text'].astype(str))
            fake_bi_freq = dict(zip(vectorizer_bi.get_feature_names_out(), fake_bigrams.sum(axis=0).A1))
            fake_bi_top = sorted(fake_bi_freq.items(), key=lambda x: x[1], reverse=True)[:10]
            
            true_bigrams = vectorizer_bi.fit_transform(df_true['text'].astype(str))
            true_bi_freq = dict(zip(vectorizer_bi.get_feature_names_out(), true_bigrams.sum(axis=0).A1))
            true_bi_top = sorted(true_bi_freq.items(), key=lambda x: x[1], reverse=True)[:10]
            
            print("\nTop 10 Bigrams (Fake News):")
            for bigram, freq in fake_bi_top:
                print(f"  {bigram}: {freq:,}")
            
            print("\nTop 10 Bigrams (True News):")
            for bigram, freq in true_bi_top:
                print(f"  {bigram}: {freq:,}")
            
            # Visualization
            fig, axes = plt.subplots(1, 2, figsize=(18, 8))
            
            fake_words, fake_counts = zip(*fake_bi_top)
            axes[0].barh(range(len(fake_words)), fake_counts, color='red', alpha=0.7)
            axes[0].set_yticks(range(len(fake_words)))
            axes[0].set_yticklabels(fake_words)
            axes[0].set_xlabel('Frequency')
            axes[0].set_title('Top 10 Bigrams - Fake News', fontweight='bold', fontsize=14)
            axes[0].invert_yaxis()
            axes[0].grid(axis='x', alpha=0.3)
            
            true_words, true_counts = zip(*true_bi_top)
            axes[1].barh(range(len(true_words)), true_counts, color='green', alpha=0.7)
            axes[1].set_yticks(range(len(true_words)))
            axes[1].set_yticklabels(true_words)
            axes[1].set_xlabel('Frequency')
            axes[1].set_title('Top 10 Bigrams - True News', fontweight='bold', fontsize=14)
            axes[1].invert_yaxis()
            axes[1].grid(axis='x', alpha=0.3)
            
            plt.tight_layout()
            plt.savefig('eda_outputs/bigrams_analysis.png', dpi=300, bbox_inches='tight')
            plt.close()
            print("✓ Saved: eda_outputs/bigrams_analysis.png")
        
        # ===== 1.5 TF-IDF ANALYSIS =====
        if 'text' in df_text.columns:
            print("\n🔍 TF-IDF Analysis...")
            
            tfidf = TfidfVectorizer(max_features=1000, stop_words='english')
            tfidf_matrix = tfidf.fit_transform(df_text['text'].astype(str))
            
            print(f"  TF-IDF matrix shape: {tfidf_matrix.shape}")
            print(f"  Vocabulary size: {len(tfidf.vocabulary_)}")
            
            # PCA visualization
            pca = PCA(n_components=2)
            tfidf_pca = pca.fit_transform(tfidf_matrix.toarray())
            
            plt.figure(figsize=(12, 8))
            plt.scatter(tfidf_pca[df_text['label'] == 0, 0], 
                       tfidf_pca[df_text['label'] == 0, 1], 
                       c='red', alpha=0.3, s=20, label='Fake')
            plt.scatter(tfidf_pca[df_text['label'] == 1, 0], 
                       tfidf_pca[df_text['label'] == 1, 1], 
                       c='green', alpha=0.3, s=20, label='True')
            plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
            plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
            plt.title('TF-IDF Feature Space (PCA)', fontweight='bold', fontsize=14)
            plt.legend()
            plt.grid(alpha=0.3)
            plt.savefig('eda_outputs/tfidf_pca.png', dpi=300, bbox_inches='tight')
            plt.close()
            print("✓ Saved: eda_outputs/tfidf_pca.png")
            
except Exception as e:
    print(f"✗ Error in text analysis: {e}")
    import traceback
    traceback.print_exc()

# =====================================================
# PHASE 2: ADVANCED IMAGE ANALYSIS
# =====================================================
print("\n" + "="*80)
print("[PHASE 2] ADVANCED IMAGE DATASET ANALYSIS")
print("="*80)

reddit_images = []
df_reddit = None

try:
    if os.path.exists(REDDIT_PATH):
        extensions = ['*.jpg', '*.jpeg', '*.png', '*.gif']
        for ext in extensions:
            reddit_images.extend(glob.glob(os.path.join(REDDIT_PATH, '**', ext), recursive=True))
        
        print(f"✓ Found {len(reddit_images)} Reddit images")
        
        # Check for metadata
        reddit_csv_files = glob.glob(os.path.join(REDDIT_PATH, '*.csv'))
        if reddit_csv_files:
            df_reddit = pd.read_csv(reddit_csv_files[0])
            print(f"✓ Reddit metadata: {len(df_reddit)} entries")
            
        # ===== 2.1 IMAGE PROPERTIES ANALYSIS =====
        if reddit_images and CV_AVAILABLE:
            print("\n📸 Detailed Image Analysis...")
            sample_size = min(200, len(reddit_images))
            sample_imgs = reddit_images[:sample_size]
            
            img_properties = {
                'widths': [], 'heights': [], 'aspects': [], 
                'sizes': [], 'channels': [], 'formats': [],
                'brightness': [], 'contrast': []
            }
            
            for i, img_path in enumerate(sample_imgs):
                if i % 50 == 0:
                    print(f"  Processing {i}/{sample_size}...", end='\r')
                try:
                    img = cv2.imread(img_path)
                    if img is not None:
                        h, w = img.shape[:2]
                        img_properties['widths'].append(w)
                        img_properties['heights'].append(h)
                        img_properties['aspects'].append(w/h if h > 0 else 0)
                        img_properties['sizes'].append(os.path.getsize(img_path)/1024)
                        img_properties['channels'].append(img.shape[2] if len(img.shape) == 3 else 1)
                        
                        # Brightness and contrast
                        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape) == 3 else img
                        img_properties['brightness'].append(np.mean(gray))
                        img_properties['contrast'].append(np.std(gray))
                        
                        # Format
                        ext = os.path.splitext(img_path)[1].lower()
                        img_properties['formats'].append(ext)
                except:
                    continue
            
            print(f"\n\n📊 Image Statistics (n={len(img_properties['widths'])}):")
            print(f"  Dimensions: {np.mean(img_properties['widths']):.0f}x{np.mean(img_properties['heights']):.0f} (avg)")
            print(f"  Aspect ratio: {np.mean(img_properties['aspects']):.2f} ± {np.std(img_properties['aspects']):.2f}")
            print(f"  File size: {np.mean(img_properties['sizes']):.1f} ± {np.std(img_properties['sizes']):.1f} KB")
            print(f"  Brightness: {np.mean(img_properties['brightness']):.1f} ± {np.std(img_properties['brightness']):.1f}")
            print(f"  Contrast: {np.mean(img_properties['contrast']):.1f} ± {np.std(img_properties['contrast']):.1f}")
            
            # Visualization
            fig, axes = plt.subplots(2, 3, figsize=(18, 12))
            
            # Width distribution
            axes[0, 0].hist(img_properties['widths'], bins=30, color='skyblue', edgecolor='black')
            axes[0, 0].set_xlabel('Width (pixels)')
            axes[0, 0].set_ylabel('Frequency')
            axes[0, 0].set_title('Image Width Distribution', fontweight='bold')
            axes[0, 0].grid(alpha=0.3)
            
            # Height distribution
            axes[0, 1].hist(img_properties['heights'], bins=30, color='lightcoral', edgecolor='black')
            axes[0, 1].set_xlabel('Height (pixels)')
            axes[0, 1].set_ylabel('Frequency')
            axes[0, 1].set_title('Image Height Distribution', fontweight='bold')
            axes[0, 1].grid(alpha=0.3)
            
            # Aspect ratio
            axes[0, 2].hist(img_properties['aspects'], bins=30, color='lightgreen', edgecolor='black')
            axes[0, 2].set_xlabel('Aspect Ratio (W/H)')
            axes[0, 2].set_ylabel('Frequency')
            axes[0, 2].set_title('Aspect Ratio Distribution', fontweight='bold')
            axes[0, 2].grid(alpha=0.3)
            
            # File size
            axes[1, 0].hist(img_properties['sizes'], bins=30, color='plum', edgecolor='black')
            axes[1, 0].set_xlabel('File Size (KB)')
            axes[1, 0].set_ylabel('Frequency')
            axes[1, 0].set_title('File Size Distribution', fontweight='bold')
            axes[1, 0].grid(alpha=0.3)
            
            # Brightness
            axes[1, 1].hist(img_properties['brightness'], bins=30, color='gold', edgecolor='black')
            axes[1, 1].set_xlabel('Brightness')
            axes[1, 1].set_ylabel('Frequency')
            axes[1, 1].set_title('Brightness Distribution', fontweight='bold')
            axes[1, 1].grid(alpha=0.3)
            
            # Contrast
            axes[1, 2].hist(img_properties['contrast'], bins=30, color='orange', edgecolor='black')
            axes[1, 2].set_xlabel('Contrast (Std Dev)')
            axes[1, 2].set_ylabel('Frequency')
            axes[1, 2].set_title('Contrast Distribution', fontweight='bold')
            axes[1, 2].grid(alpha=0.3)
            
            plt.tight_layout()
            plt.savefig('eda_outputs/image_properties.png', dpi=300, bbox_inches='tight')
            plt.close()
            print("✓ Saved: eda_outputs/image_properties.png")
            
            # Format distribution
            format_counts = Counter(img_properties['formats'])
            plt.figure(figsize=(10, 6))
            formats, counts = zip(*format_counts.most_common())
            plt.bar(formats, counts, color='teal', alpha=0.7, edgecolor='black')
            plt.xlabel('Image Format')
            plt.ylabel('Count')
            plt.title('Image Format Distribution', fontweight='bold', fontsize=14)
            plt.grid(axis='y', alpha=0.3)
            plt.savefig('eda_outputs/image_formats.png', dpi=300, bbox_inches='tight')
            plt.close()
            print("✓ Saved: eda_outputs/image_formats.png")
            
except Exception as e:
    print(f"✗ Error in image analysis: {e}")
    import traceback
    traceback.print_exc()

# =====================================================
# PHASE 3: CNN DEEP FEATURE ANALYSIS
# =====================================================
if DL_AVAILABLE and reddit_images:
    print("\n" + "="*80)
    print("[PHASE 3] DEEP CNN FEATURE EXTRACTION & ANALYSIS")
    print("="*80)
    
    try:
        print("\n🧠 Loading pre-trained models...")
        resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        
        def extract_deep_features(img_path, model):
            try:
                img = load_img(img_path, target_size=(224, 224))
                img_array = img_to_array(img)
                img_array = np.expand_dims(img_array, axis=0)
                img_array = tf.keras.applications.resnet50.preprocess_input(img_array)
                features = model.predict(img_array, verbose=0)
                return features.flatten()
            except:
                return None
        
        sample_images = reddit_images[:min(100, len(reddit_images))]
        print(f"🔍 Extracting features from {len(sample_images)} images...")
        
        features_list = []
        valid_images = []
        
        for i, img_path in enumerate(sample_images):
            if i % 20 == 0:
                print(f"  Progress: {i}/{len(sample_images)}", end='\r')
            feat = extract_deep_features(img_path, resnet_model)
            if feat is not None:
                features_list.append(feat)
                valid_images.append(img_path)
        
        if features_list:
            features_array = np.array(features_list)
            print(f"\n✓ Extracted features shape: {features_array.shape}")
            
            # Feature statistics
            print(f"\n📈 Deep Feature Statistics:")
            print(f"  Mean activation: {features_array.mean():.4f}")
            print(f"  Std activation: {features_array.std():.4f}")
            print(f"  Max activation: {features_array.max():.4f}")
            print(f"  Min activation: {features_array.min():.4f}")
            print(f"  Sparsity: {(features_array == 0).sum() / features_array.size * 100:.2f}%")
            
            # Activation distribution
            plt.figure(figsize=(14, 6))
            
            plt.subplot(1, 2, 1)
            plt.hist(features_array.flatten(), bins=100, color='purple', alpha=0.7, edgecolor='black')
            plt.xlabel('Activation Value')
            plt.ylabel('Frequency')
            plt.title('CNN Feature Activation Distribution', fontweight='bold', fontsize=12)
            plt.yscale('log')
            plt.grid(alpha=0.3)
            
            # Feature variance
            feature_vars = np.var(features_array, axis=0)
            plt.subplot(1, 2, 2)
            plt.plot(feature_vars, color='darkblue', linewidth=1)
            plt.xlabel('Feature Index')
            plt.ylabel('Variance')
            plt.title('Feature Variance Across Images', fontweight='bold', fontsize=12)
            plt.grid(alpha=0.3)
            
            plt.tight_layout()
            plt.savefig('eda_outputs/cnn_feature_distribution.png', dpi=300, bbox_inches='tight')
            plt.close()
            print("✓ Saved: eda_outputs/cnn_feature_distribution.png")
            
            # t-SNE visualization
            try:
                from sklearn.manifold import TSNE
                print("\n📊 Creating t-SNE visualization...")
                
                n_samples = min(100, len(features_array))
                perplexity_val = min(30, n_samples - 1)
                
                if n_samples > 5 and perplexity_val > 1:
                    tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity_val, n_iter=1000)
                    features_2d = tsne.fit_transform(features_array[:n_samples])
                    
                    plt.figure(figsize=(12, 10))
                    scatter = plt.scatter(features_2d[:, 0], features_2d[:, 1], 
                                        c=range(n_samples), cmap='viridis', 
                                        s=100, alpha=0.6, edgecolors='black')
                    plt.colorbar(scatter, label='Image Index')
                    plt.xlabel('t-SNE Component 1', fontsize=12)
                    plt.ylabel('t-SNE Component 2', fontsize=12)
                    plt.title('Image Feature Space Visualization (t-SNE)', fontweight='bold', fontsize=14)
                    plt.grid(alpha=0.3)
                    plt.savefig('eda_outputs/tsne_features.png', dpi=300, bbox_inches='tight')
                    plt.close()
                    print("✓ Saved: eda_outputs/tsne_features.png")
            except Exception as e:
                print(f"⚠ t-SNE visualization failed: {e}")
            
            # UMAP visualization (if available)
            try:
                from umap import UMAP
                print("\n📊 Creating UMAP visualization...")
                
                umap_model = UMAP(n_components=2, random_state=42, n_neighbors=15)
                features_umap = umap_model.fit_transform(features_array[:min(100, len(features_array))])
                
                plt.figure(figsize=(12, 10))
                scatter = plt.scatter(features_umap[:, 0], features_umap[:, 1], 
                                    c=range(len(features_umap)), cmap='plasma', 
                                    s=100, alpha=0.6, edgecolors='black')
                plt.colorbar(scatter, label='Image Index')
                plt.xlabel('UMAP Component 1', fontsize=12)
                plt.ylabel('UMAP Component 2', fontsize=12)
                plt.title('Image Feature Space Visualization (UMAP)', fontweight='bold', fontsize=14)
                plt.grid(alpha=0.3)
                plt.savefig('eda_outputs/umap_features.png', dpi=300, bbox_inches='tight')
                plt.close()
                print("✓ Saved: eda_outputs/umap_features.png")
            except:
                print("⚠ UMAP not available (install: pip install umap-learn)")
            
            # PCA analysis
            print("\n📊 PCA Analysis...")
            from sklearn.decomposition import PCA
            pca = PCA()
            pca.fit(features_array)
            
            # Explained variance
            cumsum_var = np.cumsum(pca.explained_variance_ratio_)
            n_components_95 = np.argmax(cumsum_var >= 0.95) + 1
            n_components_99 = np.argmax(cumsum_var >= 0.99) + 1
            
            print(f"  Components for 95% variance: {n_components_95}")
            print(f"  Components for 99% variance: {n_components_99}")
            
            plt.figure(figsize=(14, 6))
            
            plt.subplot(1, 2, 1)
            plt.plot(range(1, min(51, len(pca.explained_variance_ratio_)+1)), 
                    pca.explained_variance_ratio_[:50], 'b-', linewidth=2)
            plt.xlabel('Principal Component')
            plt.ylabel('Explained Variance Ratio')
            plt.title('PCA - Explained Variance per Component', fontweight='bold')
            plt.grid(alpha=0.3)
            
            plt.subplot(1, 2, 2)
            plt.plot(range(1, min(101, len(cumsum_var)+1)), cumsum_var[:100], 'r-', linewidth=2)
            plt.axhline(y=0.95, color='g', linestyle='--', label='95% variance')
            plt.axhline(y=0.99, color='b', linestyle='--', label='99% variance')
            plt.xlabel('Number of Components')
            plt.ylabel('Cumulative Explained Variance')
            plt.title('PCA - Cumulative Explained Variance', fontweight='bold')
            plt.legend()
            plt.grid(alpha=0.3)
            
            plt.tight_layout()
            plt.savefig('eda_outputs/pca_analysis.png', dpi=300, bbox_inches='tight')
            plt.close()
            print("✓ Saved: eda_outputs/pca_analysis.png")
            
    except Exception as e:
        print(f"✗ Error in CNN analysis: {e}")
        import traceback
        traceback.print_exc()

# =====================================================
# PHASE 4: CORRELATION & STATISTICAL ANALYSIS
# =====================================================
if df_text is not None:
    print("\n" + "="*80)
    print("[PHASE 4] CORRELATION & STATISTICAL ANALYSIS")
    print("="*80)
    
    try:
        # Create numerical features for correlation
        print("\n📊 Computing correlations...")
        
        numerical_features = df_text.select_dtypes(include=[np.number]).columns.tolist()
        if len(numerical_features) > 1:
            corr_matrix = df_text[numerical_features].corr()
            
            plt.figure(figsize=(12, 10))
            sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                       center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
            plt.title('Feature Correlation Matrix', fontweight='bold', fontsize=14)
            plt.tight_layout()
            plt.savefig('eda_outputs/correlation_matrix.png', dpi=300, bbox_inches='tight')
            plt.close()
            print("✓ Saved: eda_outputs/correlation_matrix.png")
        
        # Statistical tests
        from scipy import stats
        
        print("\n🔬 Statistical Tests (Fake vs True):")
        
        if 'text_length' in df_text.columns:
            # T-test for text length
            fake_lengths = df_fake['text_length']
            true_lengths = df_true['text_length']
            t_stat, p_value = stats.ttest_ind(fake_lengths, true_lengths)
            print(f"\n  Text Length T-Test:")
            print(f"    t-statistic: {t_stat:.4f}")
            print(f"    p-value: {p_value:.6f}")
            print(f"    Significant: {'Yes' if p_value < 0.05 else 'No'}")
        
        if 'word_count' in df_text.columns:
            # Mann-Whitney U test for word count
            fake_words = df_fake['word_count']
            true_words = df_true['word_count']
            u_stat, p_value = stats.mannwhitneyu(fake_words, true_words)
            print(f"\n  Word Count Mann-Whitney U Test:")
            print(f"    U-statistic: {u_stat:.4f}")
            print(f"    p-value: {p_value:.6f}")
            print(f"    Significant: {'Yes' if p_value < 0.05 else 'No'}")
        
        if 'title_length' in df_text.columns:
            # Chi-square test for categorical features
            contingency_table = pd.crosstab(df_text['has_exclamation'], df_text['label'])
            chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)
            print(f"\n  Exclamation vs Label Chi-Square Test:")
            print(f"    chi2-statistic: {chi2:.4f}")
            print(f"    p-value: {p_value:.6f}")
            print(f"    Significant: {'Yes' if p_value < 0.05 else 'No'}")
        
    except Exception as e:
        print(f"✗ Error in statistical analysis: {e}")

# =====================================================
# PHASE 5: COMPREHENSIVE SUMMARY DASHBOARD
# =====================================================
print("\n" + "="*80)
print("[PHASE 5] GENERATING COMPREHENSIVE SUMMARY")
print("="*80)

try:
    fig = plt.figure(figsize=(20, 12))
    gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
    
    # 1. Dataset sizes
    ax1 = fig.add_subplot(gs[0, 0])
    if df_text is not None:
        sizes = [len(df_fake), len(df_true), len(reddit_images)]
        labels = ['Fake Text', 'True Text', 'Images']
        colors = ['#e74c3c', '#2ecc71', '#3498db']
        ax1.bar(labels, sizes, color=colors, alpha=0.7, edgecolor='black')
        ax1.set_ylabel('Count')
        ax1.set_title('Dataset Distribution', fontweight='bold')
        ax1.grid(axis='y', alpha=0.3)
        for i, v in enumerate(sizes):
            ax1.text(i, v, f'{v:,}', ha='center', va='bottom', fontweight='bold')
    
    # 2. Text length comparison
    ax2 = fig.add_subplot(gs[0, 1])
    if df_text is not None and 'text_length' in df_text.columns:
        data = [df_fake['text_length'], df_true['text_length']]
        bp = ax2.boxplot(data, labels=['Fake', 'True'], patch_artist=True)
        bp['boxes'][0].set_facecolor('#e74c3c')
        bp['boxes'][1].set_facecolor('#2ecc71')
        ax2.set_ylabel('Characters')
        ax2.set_title('Text Length Distribution', fontweight='bold')
        ax2.grid(axis='y', alpha=0.3)
    
    # 3. Word count comparison
    ax3 = fig.add_subplot(gs[0, 2])
    if df_text is not None and 'word_count' in df_text.columns:
        data = [df_fake['word_count'], df_true['word_count']]
        bp = ax3.boxplot(data, labels=['Fake', 'True'], patch_artist=True)
        bp['boxes'][0].set_facecolor('#e74c3c')
        bp['boxes'][1].set_facecolor('#2ecc71')
        ax3.set_ylabel('Words')
        ax3.set_title('Word Count Distribution', fontweight='bold')
        ax3.grid(axis='y', alpha=0.3)
    
    # 4. Title characteristics
    ax4 = fig.add_subplot(gs[1, 0])
    if df_text is not None and 'has_question' in df_text.columns:
        fake_q = (df_fake['has_question'].sum() / len(df_fake) * 100)
        true_q = (df_true['has_question'].sum() / len(df_true) * 100)
        fake_e = (df_fake['has_exclamation'].sum() / len(df_fake) * 100)
        true_e = (df_true['has_exclamation'].sum() / len(df_true) * 100)
        
        x = np.arange(2)
        width = 0.35
        ax4.bar(x - width/2, [fake_q, fake_e], width, label='Fake', color='#e74c3c', alpha=0.7)
        ax4.bar(x + width/2, [true_q, true_e], width, label='True', color='#2ecc71', alpha=0.7)
        ax4.set_ylabel('Percentage (%)')
        ax4.set_title('Punctuation in Titles', fontweight='bold')
        ax4.set_xticks(x)
        ax4.set_xticklabels(['Questions', 'Exclamations'])
        ax4.legend()
        ax4.grid(axis='y', alpha=0.3)
    
    # 5. Image properties (if available)
    ax5 = fig.add_subplot(gs[1, 1])
    if reddit_images and CV_AVAILABLE:
        sample_imgs = reddit_images[:50]
        sizes = []
        for img_path in sample_imgs:
            try:
                sizes.append(os.path.getsize(img_path)/1024)
            except:
                continue
        if sizes:
            ax5.hist(sizes, bins=20, color='#3498db', alpha=0.7, edgecolor='black')
            ax5.set_xlabel('File Size (KB)')
            ax5.set_ylabel('Frequency')
            ax5.set_title('Image File Sizes', fontweight='bold')
            ax5.grid(axis='y', alpha=0.3)
    
    # 6. Data quality metrics
    ax6 = fig.add_subplot(gs[1, 2])
    if df_text is not None:
        metrics = {
            'Text Samples': len(df_text),
            'Image Samples': len(reddit_images),
            'Missing Values': df_text.isnull().sum().sum(),
            'Duplicates': df_text.duplicated().sum()
        }
        ax6.barh(list(metrics.keys()), list(metrics.values()), color='#9b59b6', alpha=0.7)
        ax6.set_xlabel('Count')
        ax6.set_title('Data Quality Metrics', fontweight='bold')
        ax6.grid(axis='x', alpha=0.3)
    
    # 7. Class balance
    ax7 = fig.add_subplot(gs[2, 0])
    if df_text is not None:
        class_counts = df_text['label'].value_counts()
        colors = ['#e74c3c', '#2ecc71']
        ax7.pie(class_counts.values, labels=['Fake', 'True'], autopct='%1.1f%%',
               colors=colors, startangle=90)
        ax7.set_title('Class Balance', fontweight='bold')
    
    # 8. Summary statistics table
    ax8 = fig.add_subplot(gs[2, 1:])
    ax8.axis('tight')
    ax8.axis('off')
    
    if df_text is not None:
        summary_data = []
        summary_data.append(['Total Articles', f"{len(df_text):,}"])
        summary_data.append(['Fake Articles', f"{len(df_fake):,}"])
        summary_data.append(['True Articles', f"{len(df_true):,}"])
        summary_data.append(['Total Images', f"{len(reddit_images):,}"])
        if 'text_length' in df_text.columns:
            summary_data.append(['Avg Text Length (Fake)', f"{df_fake['text_length'].mean():.0f} chars"])
            summary_data.append(['Avg Text Length (True)', f"{df_true['text_length'].mean():.0f} chars"])
        
        table = ax8.table(cellText=summary_data, colLabels=['Metric', 'Value'],
                         cellLoc='left', loc='center',
                         colWidths=[0.5, 0.5])
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1, 2)
        
        for i in range(len(summary_data) + 1):
            if i == 0:
                table[(i, 0)].set_facecolor('#34495e')
                table[(i, 1)].set_facecolor('#34495e')
                table[(i, 0)].set_text_props(weight='bold', color='white')
                table[(i, 1)].set_text_props(weight='bold', color='white')
            else:
                table[(i, 0)].set_facecolor('#ecf0f1')
                table[(i, 1)].set_facecolor('#ecf0f1')
    
    plt.suptitle('Comprehensive EDA Summary Dashboard', fontsize=18, fontweight='bold', y=0.98)
    plt.savefig('eda_outputs/summary_dashboard.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("✓ Saved: eda_outputs/summary_dashboard.png")
    
except Exception as e:
    print(f"✗ Error creating summary dashboard: {e}")
    import traceback
    traceback.print_exc()

# =====================================================
# FINAL SUMMARY
# =====================================================
print("\n" + "="*80)
print("✅ ADVANCED EDA COMPLETED")
print("="*80)

print("\n📁 Generated Outputs (in eda_outputs/):")
output_files = [
    'text_length_analysis.png',
    'title_analysis.png',
    'wordclouds.png',
    'bigrams_analysis.png',
    'tfidf_pca.png',
    'image_properties.png',
    'image_formats.png',
    'cnn_feature_distribution.png',
    'tsne_features.png',
    'pca_analysis.png',
    'correlation_matrix.png',
    'summary_dashboard.png'
]

for filename in output_files:
    filepath = os.path.join('eda_outputs', filename)
    if os.path.exists(filepath):
        print(f"  ✓ {filename}")

print("\n🔬 Analysis Components Completed:")
print("  ✓ Text length & word count analysis")
print("  ✓ Title characteristics (punctuation, caps)")
print("  ✓ Word clouds (fake vs true)")
print("  ✓ N-gram analysis (bigrams)")
print("  ✓ TF-IDF feature extraction & PCA")
print("  ✓ Image properties (dimensions, brightness, contrast)")
print("  ✓ Deep CNN feature extraction (ResNet50)")
print("  ✓ t-SNE & UMAP visualizations")
print("  ✓ PCA dimensionality analysis")
print("  ✓ Statistical hypothesis testing")
print("  ✓ Correlation analysis")
print("  ✓ Comprehensive summary dashboard")

print("\n📊 Key Insights:")
if df_text is not None:
    print(f"  • Dataset balance: {len(df_fake):,} fake vs {len(df_true):,} true articles")
    if 'text_length' in df_text.columns:
        fake_avg = df_fake['text_length'].mean()
        true_avg = df_true['text_length'].mean()
        diff_pct = abs(fake_avg - true_avg) / true_avg * 100
        print(f"  • Text length difference: {diff_pct:.1f}%")
    if 'has_exclamation' in df_text.columns:
        fake_exc = (df_fake['has_exclamation'].sum() / len(df_fake) * 100)
        true_exc = (df_true['has_exclamation'].sum() / len(df_true) * 100)
        print(f"  • Exclamation usage: {fake_exc:.1f}% (fake) vs {true_exc:.1f}% (true)")

print("\n🎯 Next Steps:")
print("  1. Feature engineering based on EDA insights")
print("  2. Model selection (CNN for images, LSTM/Transformer for text)")
print("  3. Multimodal fusion architecture design")
print("  4. Training with proper validation strategy")

print("\n" + "="*80)

🔧 Checking dependencies...

🔬 ADVANCED EDA - MULTIMODAL FAKE NEWS DETECTION

[PHASE 1] ADVANCED TEXT DATA ANALYSIS
✓ Loaded 44,898 articles (23,481 fake, 21,417 true)
  Columns: ['title', 'text', 'subject', 'date', 'label', 'category']

📏 Text Length Analysis:

Fake News Statistics:
  Avg length: 2547 chars
  Avg words: 423

True News Statistics:
  Avg length: 2383 chars
  Avg words: 386
✓ Saved: eda_outputs/text_length_analysis.png

📰 Title Analysis:

Fake News Titles:
  Avg length: 94.2 chars
  Questions: 7.0%
  Exclamations: 13.9%

True News Titles:
  Avg length: 64.7 chars
  Questions: 0.6%
  Exclamations: 0.1%
✓ Saved: eda_outputs/title_analysis.png

☁️ Generating Word Clouds...
✓ Saved: eda_outputs/wordclouds.png

📊 N-Gram Analysis...

Top 10 Bigrams (Fake News):
  donald trump: 16,288
  featured image: 7,692
  hillary clinton: 7,278
  white house: 6,743
  united states: 6,663
  twitter com: 6,567
  pic twitter: 6,195
  new york: 4,352
  president obama: 4,081
  getty images: 4,0